## Simulate Ultrasound Image from CT slice example

In [1]:
import cv2
import time
import torch
import numpy as np
import torch.nn.functional as F

from unet.model import UNet_dilation, UNet, AttentionUNet
from utils.vis import array2tensor, tensor2array

### Make sure GPU is available

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

Using device cuda


### load pretrained network

In [39]:
net = AttentionUNet(n_channels=1, n_classes=1, bilinear=False)
net.to(device=device)
net.load_state_dict(torch.load('model/ct2us_attention_unet_cur.pth'))
# print(net.eval())

<All keys matched successfully>

### Load example CT slice 

In [43]:
ct_raw = cv2.imread('../dataset/ct_slices_trans_06-Mar-2024/ct_msk/ctmsk_56.png', cv2.IMREAD_GRAYSCALE)
print(ct_raw.shape)
ct_raw = cv2.normalize(ct_raw, None, 1, 0, cv2.NORM_MINMAX)

(215, 286)


### Simulate Ultrasound from CT slice

In [44]:
start = time.perf_counter()

ct = array2tensor(ct_raw, device=device)
print(f'input shape: {ct.shape}')

pred = net(ct)
us_sim = tensor2array(pred)*255
us_sim 

print(f'prediction shape: {pred.shape}')
print(f'time elapsed: {(time.perf_counter()-start):.3f} sec')  # benchmarking

cv2.imwrite('test_us_sim_out.png', us_sim)

input shape: torch.Size([1, 1, 215, 286])
prediction shape: torch.Size([1, 1, 215, 286])
time elapsed: 0.040 sec


True